In [ ]:
import pandas as pd
import os
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn' or None

In [ ]:
# The first step is to set up the folder variables for where to read and write data. Since the dataset is gigantic the downloading was done
# outside of Python.

dataset_folder = r'C:\Users\mpola\OneDrive\Desktop\Career\Proje\DREAMT\dataset'

grouped_data_folder = dataset_folder + '\\grouped_100Hz'
clean_data_folder = dataset_folder + '\\clean_data'

if not os.path.exists(grouped_data_folder):
    os.makedirs(grouped_data_folder)

if not os.path.exists(clean_data_folder):
    os.makedirs(clean_data_folder)

df_participants = pd.read_csv(f'{dataset_folder}\\participant_info.csv')

Custom Functions
---

In [ ]:
##################################################
# For more readability, we define and use a function to format all our column names and string entries such that 
# they're properly and consistently capitalized. This function does not affect all-uppercase words as the majority of these are medical acronyms
def format_text(text):
    text = text.replace('_', ' ')
    temp_text = ''
    for i, letter in enumerate(text):
        if i == 0 or text[i-1] == ' ':
            temp_text = temp_text + letter.capitalize()
        else:
            temp_text = temp_text + letter
    return temp_text
##################################################
# Function to iterate format_text on every column name
def format_cols(df):
    return df.rename(mapper=format_text, axis='columns')
##################################################

In [ ]:
##################################################
def table_stack(df, id_col, col_name):
# This function is a simple stacking task to help automate the compression of a multitude of boolean columns into one column that counts the occurence of
# that column alongside the index column. This is much more effective than storing dozens of boolean columns, as we have a lot of rows that have only one
# boolean value and a ton of redundant Falses.

    df_temp = df.set_index(id_col).stack().reset_index()
    
    df_temp.columns = [id_col, col_name, 'bool_val_tmp']
    
    df_stacked = df_temp[df_temp['bool_val_tmp']==1]
    
    return df_stacked.drop(labels='bool_val_tmp', axis=1)
##################################################

In [ ]:
# The participants info CSV file has two problematic columns: MEDICAL_HISTORY and Sleep_Disorders. These columns store info regarding patients
# in a string format, where certain conditions show up separated by commas. This makes the data annoying to work with in external programs, so
# we define a function to separate these string columns into boolean columns using a function.
# These functions are imported from a previous portfolio project, Stack Overflow 2022-2024 Surveys Analysis, which can be found in the below Github link:

# https://github.com/MustafaBerkPolat/so22-24survey

##################################################
def split_string_checklist(df, column_name, separator=';', identifier='null'):
# This function takes a string column of a dataframe and splits this column into multiple boolean columns based on
# the occurence of a separator character, most typically a semicolon, and outputs a new dataframe containing these new
# bool columns
# The identifier input, if not left to its default value of 'null', adds a prefix before the column names to help identify shared column
# names between dataframes

# Making array from columns and discarding repeat entries to make it easier to work with
    arr = df[column_name].unique()

# Splitting values and making nested list with them
    nested_lists = [str(i).split(separator) for i in arr]


# Getting all the unique values from list (getting the new column names)
    split_values = set()
    
    for sublist in nested_lists:
        split_values.update(sublist)
    split_values = sorted(split_values)

    
# Making new df with boolean, isinstance to avoid null/nan values
    new_df = pd.DataFrame(df[column_name])
    for column in split_values:
        new_df[column] = df[column_name].apply(
            lambda x: column in x if isinstance(x, (list, str)) else False).astype(int)

    
# Replacing null values with True bool and non-null values with False bool for selected column for future analysis
    df_temp = pd.DataFrame(df)
    df_temp[column_name] = df_temp[column_name].apply(lambda x: True if pd.isnull(x) else False)

    
# Combining 'duplicate' columns with different notations into one column, since not every name is consistently formatted across years
# eg. 'Couch DB' is spelled as 'CouchDB' in one year
    simple_names = []
    names= []

    for col in split_values:
        names.append(col)
        simple_names.append(col.replace(" ", "").replace("-", "").replace("_", "").lower())

    
    pairs = []

    for i,name in enumerate(names):
        pairs.append([name, names[simple_names.index(simple_names[i])]])

    
    for col in pairs:
        if col[0] != col[1]:
            new_df[col[1]] = new_df[[col[0], col[1]]].any(axis=1).astype(int)

# Dropping the now unnecessary duplicate columns, and renaming the remaining column to include the optional identifier string
# at the start of the column names. This is useful as multiple dataframes can have columns that have the same name but represent
# different things
    for col in pairs:
        if col[0] != col[1]:
            if col[0] in new_df.columns:
                new_df.drop(col[0], axis=1, inplace=True)
        else:
            if identifier != 'null':
                new_col = identifier + '-' + col[1]
                new_df[new_col] = new_df[col[1]]
                new_df.drop(col[1], axis=1, inplace=True)

        
# Combining new df with old one
    df_combined = pd.concat([df_temp*1, new_df], axis=1)
    df_combined.drop(column_name, axis=1, inplace=True)

    if 'nan' in df_combined.columns:
        df_combined.drop('nan', axis=1, inplace=True)

    
    return df_combined
##################################################

Aggregating the measured data
---

In [ ]:
# The raw data we have totals to over 3 billion rows and 30 columns, so it needs to be aggregated to be workable first. Since the data has sleep stage
# tracking, doing this aggregation by grouping by both participant and sleep stage is the most handy aggregation we can calculate

files_list_raw = os.listdir(dataset_folder + '\\data_100Hz')
# files_list_raw = ['S002_PSG_df.csv']

# The aggregate functions we will be calculating. We calculate the minimum and maximum for the variable measurements, but for the columns that track
# whether an event has occurred or not, tracking these values is pointless and can be replaced by the sum of these incidents instead.
aggs = ['mean', 'std', 'min', 'max']
counts = ['mean', 'std', 'sum']


agg_funcs = {
    'C4-M1': aggs,
    'F4-M1': aggs,
    'O2-M1': aggs,
    'Fp1-O2': aggs,
    'T3 - CZ': aggs,
    'CZ - T4': aggs,
    'CHIN': aggs,
    'E1': aggs,
    'E2': aggs,
    'ECG': aggs,
    'LAT': aggs,
    'RAT': aggs,
    'SNORE': aggs,
    'PTAF': aggs,
    'FLOW': aggs,
    'THORAX': aggs,
    'ABDOMEN': aggs,
    'SAO2': aggs,
    'BVP': aggs,
    'ACC_X': aggs,
    'ACC_Y': aggs,
    'ACC_Z': aggs,
    'TEMP': aggs,
    'EDA': aggs,
    'HR': aggs,
    'IBI': aggs,
    'Obstructive_Apnea': counts,
    'Central_Apnea': counts,
    'Hypopnea': counts
}


# Defining the dataframe that'll contain the aggregated info
df_aggregate = pd.DataFrame()

########################################################

for index, file in enumerate(files_list_raw):
    df_temp = pd.read_csv(f'{dataset_folder}\\data_100Hz\\{file}')
    
########################################################
    
    # We create a separate copy of the read dataframe to perform a separate aggregation operation on. This uses more memory but is much faster than
    # reading the file twice in my system, and allows us to calculate the duration of each sleep stage more easily to then append to our main
    # aggregated dataframe.
    df_temp_duration = df_temp[['TIMESTAMP', 'Sleep_Stage']]

    
    # Reading the SID column of the participant info CSV to append to the aggregate measurements data
    df_temp_duration['SID'] = df_participants.iloc[index,0]


    # Calculating the individual sleep stages. We can't simply use groupby since these sleep stages repeat throughout the night with other stages
    # interspersed between them, and we want to account for these separations and treat each repetition of a stage separately for aggregation
    df_temp_duration['Stage_Number'] = (df_temp_duration['Sleep_Stage'] != df_temp_duration['Sleep_Stage'].shift()).cumsum()


    # Calculating how long each sleep stage is
    duration = df_temp_duration.groupby('Stage_Number')['TIMESTAMP'].apply(lambda x: x.max() - x.min())


    # Grouping by the stage number value to then assign the stage duration parameters for
    df_temp_duration = df_temp_duration.groupby('Stage_Number').agg({
        'TIMESTAMP': 'std',
        'Sleep_Stage': lambda x: x.mode()[0]
    })

    
    # Breaking the multi-index structure and renaming the columns
    # df_temp_duration.rename(columns={'Sleep Stage <lambda>': 'Sleep Stage'}, inplace=True)

    
    # Defining the Stage_Duration column with our previous calculations
    df_temp_duration['Stage_Duration'] = duration.values


    # Aggregating the stage duration per different kind of sleep stage and counting how many times each stage occurs
    df_temp_duration = df_temp_duration.groupby('Sleep_Stage').agg({
        'Stage_Duration': aggs,
        'Sleep_Stage': 'count'})

    # Flattening the MultiIndex columns to be exportable
    df_temp_duration.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_temp_duration.columns.values]

########################################################
    
    # Then we move on to aggregating the actual measured information like heart rate and electrodermal activity. 
    df_temp = df_temp.fillna(0).groupby('Sleep_Stage').agg(agg_funcs)

    
    # Flattening the MultiIndex columns to be exportable
    df_temp.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_temp.columns.values]


    # Merging the two dataframes together
    df_temp = pd.merge(df_temp, df_temp_duration, on='Sleep_Stage', how='outer')


    # Reading the SID column of the participant info CSV to append to the aggregate measurements data
    df_temp['SID'] = df_participants.iloc[index,0]


    # Appending the merged dataframes back into the main aggregate dataframe
    df_aggregate = pd.concat([df_aggregate, df_temp])

########################################################

In [ ]:
df_aggregate.reset_index(drop=False, inplace=True)


# Reordering columns for clarity
df_aggregate = df_aggregate[['SID', 'Sleep_Stage'] + [col for col in df_aggregate.columns if col not in ['SID', 'Sleep_Stage']]]


# Formatting the columns to be more readable. This function leaves some of the measurement data as capitalized (like THORAX and CHIN) but
# this distinction makes it easier to differentiate between measurements and similarly named conditions so it is not worth it to try and rename them
# in this case
df_aggregate = format_cols(df_aggregate)


# Renaming the columns with redundant spaces to get rid of these spaces so that they're easier to format alongside our other columns
# when visualizing
df_aggregate.rename(columns={
     'T3 - CZ Mean': 'T3-CZ Mean',
     'T3 - CZ Std': 'T3-CZ Std',
     'T3 - CZ Min': 'T3-CZ Min',
     'T3 - CZ Max': 'T3-CZ Max',
     'CZ - T4 Mean': 'CZ-T4 Mean',
     'CZ - T4 Std': 'CZ-T4 Std',
     'CZ - T4 Min': 'CZ-T4 Min',
     'CZ - T4 Max': 'CZ-T4 Max'}, inplace=True)


# Saving the file
df_aggregate.to_csv(f'{clean_data_folder}\\df_aggregate.csv', index=False)

Cleaning the participant info CSV and creating pivot-table style CSV files for the medical history and sleep condition data
---

In [ ]:
# By assigning an identifier to separate the end results of the splitting of two columns, we ensure there is no confusion regarding
# where each column comes from. This is needed as conditions like 'sleep apnea' can show up in both and knowing which column the data
# came from is important as the Sleep_Disorder column tracks the researchers' observation during the experiment, while MEDICAL_HISTORY
# represents past diagnoses.

df_medical_history = split_string_checklist(df_participants, 'MEDICAL_HISTORY', separator=', ')
df_sleep_disorders = split_string_checklist(df_participants, 'Sleep_Disorders', separator=', ')

In [ ]:
# Combining 'grinds teeth' and 'grind teeth' under 'bruxism', the scientific term for teeth grinding
df_sleep_disorders['bruxism'] = df_sleep_disorders[['grinds teeth', 'grind teeth', 'bruxism']].any(axis='columns').astype(int)

df_sleep_disorders.drop(labels=['grind teeth', 'grinds teeth'], axis='columns', inplace=True)



# Combining the 'fatigue' and 'chronic fatigue' columns under 'chronic fatigue'
df_sleep_disorders['Chronic fatigue'] = df_sleep_disorders[['fatigue', 'Chronic fatigue']].any(axis='columns').astype(int)

df_sleep_disorders.drop(labels='fatigue', axis='columns', inplace=True)



# Dismantling the 'OSA snoring' column into the 'OSA' and 'Snoring' columns, also combining 'Snoring' and 'snore
df_sleep_disorders['OSA'] = df_sleep_disorders[['OSA', 'OSA snoring']].any(axis='columns').astype(int)
df_sleep_disorders['Snoring'] = df_sleep_disorders[['Snoring', 'OSA snoring', 'snore']].any(axis='columns').astype(int)

df_sleep_disorders.drop(labels=['OSA snoring', 'snore'], axis='columns', inplace=True)



# Combining the 'diffifulty breathing' column into the one without typos
df_sleep_disorders['difficulty breathing'] = df_sleep_disorders[['difficulty breathing', 'diffifulty breathing']].any(axis='columns').astype(int)

df_sleep_disorders.drop(labels='diffifulty breathing', axis='columns', inplace=True)



# Combining 'snort' and 'snorts'
df_sleep_disorders['snort'] = df_sleep_disorders[['snort', 'snorts']].any(axis='columns').astype(int)

df_sleep_disorders.drop(labels='snorts', axis='columns', inplace=True)



# Combining all the different variations of sleep apnea under one sleep disorder column, and splitting the 'MCI and sleep apnea' column into its own
# MCI column. Everyone in this research who was reported to have major cognitive impairment also had sleep apnea, so we can just 
# rename the existing column into MCI and include everyone in it in our sleep apnea column as well without problem
df_sleep_disorders['Sleep apnea'] = df_sleep_disorders[[
    'MCI and Sleep apnea',
    'OSA', 
    'H/O OSA',
    'Sleep apnea']].any(axis='columns').astype(int)



df_sleep_disorders.drop(labels=['OSA', 'H/O OSA'], axis='columns', inplace=True)
df_sleep_disorders.rename(columns={'MCI and Sleep apnea': 'MCI'}, inplace=True)



# Since 'Sleep Apnea' is present in both the sleep disorders and medical history dataframes, we need to either rename one and keep both, or
# combine the two after merging the dataframes back into our participants frame. In this case, I opted for renaming the sleep disorders entry to
# 'Sleep Apnea (Observed)'
df_sleep_disorders.rename(columns={'Sleep apnea': 'Sleep Apnea (Observed)'}, inplace=True)

In [ ]:
# We can drop the columns that correspond to no as a response as well as the columns that contain non-boolean data that will be
# stored in a separate CSV file
df_sleep_disorders.drop(labels=['AGE', 'GENDER', 'BMI', 'OAHI', 'AHI', 'Mean_SaO2',  'Arousal Index', 'MEDICAL_HISTORY', 'none'], axis='columns', inplace=True)

df_medical_history.drop(labels=['AGE', 'GENDER', 'BMI', 'OAHI', 'AHI', 'Mean_SaO2',  'Arousal Index', 'Sleep_Disorders'], axis='columns', inplace=True)


# Formatting the datasets to be more readable with our format_cols function
df_sleep_disorders = format_cols(df_sleep_disorders)
df_medical_history = format_cols(df_medical_history)

In [ ]:
# Since the participants and grouped dataframe has some columns that make sense to be capitalized (BMI, OAHI, SID etc.) and others that don't 
# (AGE and GENDER), it makes sense to do manual work here as opposed to working out the edge cases of our functions
df_participants.rename(columns={
    'AGE': 'Age', 
    'GENDER': 'Gender',
    'Mean_SaO2': 'Mean SaO2'}, inplace=True)

In [ ]:
# Finally, we stack the boolean columns back together to have a pivot table-style CSV file for the sleep disorder and medical history info.
# Before this stacking, we can also re-merge the boolean columns into our participants info table, just so we have the data in the
# boolean columns format as well, in case this format is easier to use in our analysis program than the pivot table-style.

# Dropping the hard-to-work-with string columns
df_participants.drop(labels=['MEDICAL_HISTORY', 'Sleep_Disorders'], axis='columns', inplace=True)


# Merging the boolean sleep disorder and medical history dataframes together to then merge with the participant info
df_temp = pd.merge(df_sleep_disorders, df_medical_history, on='SID', how='outer')
df_participants = pd.merge(df_participants, df_temp, on='SID', how='outer')


# Stacking the sleep disorder and medical history dataframes into the pivot table format
df_sleep_disorders = table_stack(df_sleep_disorders, 'SID', 'Sleep Disorders')
df_medical_history = table_stack(df_medical_history, 'SID', 'Medical History')

In [ ]:
# Saving the three dataframes
df_participants.to_csv(f'{clean_data_folder}\\df_participants.csv', index=False)

df_sleep_disorders.to_csv(f'{clean_data_folder}\\df_sleep_disorders.csv', index=False)

df_medical_history.to_csv(f'{clean_data_folder}\\df_medical_history.csv', index=False)